In [ ]:
#used the Typing to make the code more readable

In [2]:
from random import choices, randint, randrange, random
from typing import List, Callable, Tuple
import time
from Agent import Agent
from tqdm import tqdm
from Simulation import example_simulation
from Obstacle import Obstacle

In [3]:
Chromosome = List[int]
Population = List[Chromosome]
Init_population = Callable[[int,int],Population]
Fitness_function = Callable[[Chromosome],int] #this is a function that asses the validity of our solution
Mutation_operator = Callable[[Chromosome,float],Chromosome]
Crossover_operator = Callable[[Chromosome,Chromosome],Tuple[Chromosome,Chromosome]] #Create off spring Chromosomes 

In [4]:
#a function used to generate randomly a solution
def generate_chromosome(length: int) -> Chromosome:
    return choices([0, 1], k=length)
#print(generate_chromosome(6))
#this function would be used to intialise a varied initial population
def init_population(size,length) -> Population:
    return [generate_chromosome(length) for _ in range(size)]
#print(init_population(50,5))

Function used to bidge the agent-based model and the GA

In [5]:
'''
a function that returns the coodinates of an 
obstacle given its index in the search grid ( descritized into nxn small cubes of height h)
Only works for square discretization of the search space (n = m ) 
with n > 1 

Tested
'''
def chromosome_to_index(chromosome: Chromosome) -> int:
    chromosome_string = ''.join([str(_) for _ in chromosome])
    return int(str(chromosome_string),2)+1
#print(chromosome_to_index([1,1,1,1,1]))

def index_to_position(index:int,origin_coodinates,n,pas) -> Tuple:
    O = origin_coodinates
    if index == 0:
        return O
    elif index % n == 0 : 
        (x,y) = (O[0]+(n-1)*pas,O[1]+(index//n -1) * pas )
    else:
        (x,y) = (O[0] + (index % n -1)* pas,O[1] + index//n * pas) #cette formule fonctionne pour tout les tetes sauf pour celle ou le reste est nul raison pour quelle on a ajouté la condition au dessus
    return (x,y)
# add an out of index function
#index_to_position(5,(0,0),4,1) #testing in a grid of size(4x4) with a step of 1

# a function that returns the decimal index of a chromome from its binary encoding

def chromosome_to_1_obstacle(chromosome) -> Obstacle: #in case one obstacle would be placed
    index = chromosome_to_index(chromosome)
    position = index_to_position(index,(60,40),8,3)
    return [Obstacle(position,10,10)]
def chromosome_to_2_obstacles(chromosome) -> Obstacle: #in case two obstacle would be placed
    index1 = chromosome_to_index(chromosome[:6])
    position1 = index_to_position(index1,(60,40),8,3)
    index2 = chromosome_to_index(chromosome[6:])
    position2 = index_to_position(index2,(60,40),8,3)
    return [Obstacle(position1,5,5),Obstacle(position2,5,5)]
#print(chromosome_to_1_obstacle([1,1,1,1,1]))
#should return how fit the solution is, for the time being, this one returns simulation time 
def fitness_function(chromosome:Chromosome) -> float:
    obstacle = chromosome_to_1_obstacle(chromosome)
    try:
        return (example_simulation(obstacle)) #this function returns the simulation time
    except:
        #print('something might have been wrong with this configuration')
        return 40 # The simulation breaks for some obstacle's configuration, in case it did the maximum simulation time is to be given  

In [7]:
def population_information(population:Population,fitness_function:Fitness_function):
    output = []
    for _ in tqdm(range(len(population)),desc = 'Calculating population fitness'): 
        chromosome_information = {'chromosome':population[_],'fitness':fitness_function(population[_])}
        output.append(chromosome_information)
    return output
def sort_population(population_information):
    print(sorted(Result, key=lambda k: k['fitness'],reverse = True))

Calculating population fitness: 100%|████████████████████████████████████████████████████| 3/3 [00:48<00:00, 16.25s/it]

[{'chromosome': [0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0], 'fitness': 60.0}, {'chromosome': [1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0], 'fitness': 80}, {'chromosome': [1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0], 'fitness': 39.400000000000006}]


In [8]:
#example 
population = init_population(3,6) #case for one obstacle encoding, 64 index ( 8x8 grid)

population_information = population_information(population,fitness_function)




Calculating population fitness:   0%|                                                            | 0/3 [00:00<?, ?it/s]


ValueError: invalid literal for int() with base 2: ''

In [12]:
def single_point_crossover(a: Chromosome, b: Chromosome) -> Tuple[Chromosome, Chromosome]:
    length = len(a)
    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]
#print(single_point_crossover([1,0,1,1],[0,0,0,0]))

([1, 0, 0, 0], [0, 0, 1, 1])


In [28]:
#chooses randomly 2 parents,yet the choice is biased by the fitness of the solutions, hence, the fittest a solution, the more likely it would be chosen  
def wheel_parent_selector(population: Population, f
                          itness_function: Fitness_function) -> Population: #population in the sens of being a list of 2 chromosomes
    return choices(
        population,
        weights=[int(Fitness_function(chromosome)) for chromosome in population],
        k=2
    )

SyntaxError: invalid syntax (<ipython-input-28-310e241ed069>, line 3)

In [29]:
def mutation_operator(chromosome : Chromosome, p = .1) -> Chromosome:
    if random() <= p:  # Mutation happening with a probability of p
        mutated_gene = randrange(len(chromosome))
        chromosome[mutated_gene] = abs(chromosome[mutated_gene]-1)
        return chromosome
    return chromosome
#for i in range(10):
#    print(mutation_operator([0,1,1],.1)) #example of a mutation happening with a probability of 10%

Couplage Modèle de Mobilité - Algorithm génetiques :

In [31]:
def print_stats(population: Population, generation_id: int, fitness_function: Fitness_function):
    print("GENERATION %02d" % generation_id)
    print("=============")
    #print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]


In [9]:
def Run_evolution(
    population: Population,
    init_population : Init_population,
    single_point_crossover : Crossover_operator,
) -> Population:
    for i in range(generation_limit):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-9-b0c5e941be4d>, line 7)